In [7]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import boxcox

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
# frequency table of left
turnover['left'].value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [9]:
# changing sales and salary dummies
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis =  1 ), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [11]:
# defining scaler 
scaler = MinMaxScaler()

#changing scale 0-1
turnover[['number_project', 'average_montly_hours']] = scaler.fit_transform(turnover[['number_project', 'average_montly_hours']])

# box cox transformation
transformed_time_spend = boxcox(turnover['time_spend_company'])
turnover['time_spend_company'] = transformed_time_spend[0]

turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,0.0,0.285047,0.804651,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,0.6,0.775701,1.098118,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,1.0,0.822430,0.941381,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,0.6,0.593458,1.032330,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,0.0,0.294393,0.804651,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [12]:
# defining imput and target
x = turnover.drop(columns = 'left', axis = 1)
y = turnover['left']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [15]:
# Random Forest model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(x_train, y_train)

# predicting on test
rf_pred = rf_md.predict_proba(x_test)[:,1]

# likilyhoods to labels
rf_label = np.where(rf_pred < 0.22, 0, 1)

print(classification_report(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      2286
           1       0.64      0.95      0.76       714

    accuracy                           0.86      3000
   macro avg       0.81      0.89      0.83      3000
weighted avg       0.90      0.86      0.87      3000



In [17]:
# ada boost model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = .01).fit(x_train, y_train)

# predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

# likilyhoods to labels
ada_label = np.where(ada_pred < 0.22, 0, 1)

print(classification_report(y_test, ada_label))

              precision    recall  f1-score   support

           0       1.00      0.32      0.49      2286
           1       0.32      1.00      0.48       714

    accuracy                           0.48      3000
   macro avg       0.66      0.66      0.48      3000
weighted avg       0.84      0.48      0.49      3000



In [ ]:
# based on the classification report, i would use the random forest model to predict left